# Teste: Consumo de Dados da Api do Spotify para analises sobre artistas e músicas.

## Objetivo

Avaliar conhecimentos nas linguagens Python e SQL e na engine de processamento Apache Spark.

## Descrição

Dado um conjunto de dados de músicas e artistas disponíveis em uma API pública, implemente um pipeline de processamento de dados com PySpark que realiza as seguintes operações:

[x] Criar credenciais da API

[ ] Consuma a API e extraia as informações de músicas e artistas dos seguintes genêros("Rock Nacional", "Piseiro/Arrocha" e "Pop Internacional");

[ ] Armazene os dados em formato parquet, particionando por artista;

[ ] Crie um dataframe com o endpoint "Get Artist"

[ ] Crie um dataframe com o endpoint "Get Artist's Albums" e traga as músicas dos álbuns que estão no endpoint "Get Album Tracks".

[ ] Crie um dataframe com o endpoint "Get Current User's Playlists"

[ ] Crie uma tabela temporária em PySpark a partir do DataFrame de músicas dos artistas;

[ ] Crie uma tabela temporária em Pyspark a partir do Dataframe de Playlists

[ ] Execute uma consulta SQL que retorna os artistas do endpoint "Get Artist" que estão nas Playlists, ordenados por ordem alfabética.

[ ] Crie um Dataframe com o resultado da consulta e salve em parquet

.

-------

Documentação da Api para consulta:
```
https://developer.spotify.com/documentation/web-api/
```

Para criar um Access Token na Spotify API, você precisa seguir os seguintes passos:

Acesse o Dashboard da API Spotify:
```
https://developer.spotify.com/dashboard/login
```
1. Faça login com sua conta do Spotify ou crie uma nova conta, caso ainda não tenha.
2. Crie um novo aplicativo clicando no botão "Create an App" e preencha as informações necessárias.
3. Após criar o aplicativo, você será direcionado para a página do aplicativo, onde poderá encontrar sua Client ID e Client Secret. Anote essas informações, pois elas serão necessárias para a autenticação.
4. Para gerar o Access Token, você precisará fazer uma solicitação GET/POST para o endpoint de autorização da API, passando sua Client ID e Client Secret como parâmetros. Você pode usar ferramentas como o Postman para realizar essa solicitação.
5. O endpoint de autorização irá retornar um Access Token que você pode utilizar para fazer requisições à API Spotify.

------------


## Atenção

- Leia a documentação dos endpoints para a extração dos dados, em alguns podem possuir número máximo de itens a serem retornados.

### Para realizar esta tarefa, os candidatos devem ter conhecimento em:

- PySpark (Spark SQL, Spark DataFrames);
- Consumo de API REST;
- armazenamento de dados;
- Transformações e consultas SQL em PySpark.
-------------
## Entregando o desafio

Faça uma cópia do desafio antes de começar a fazer o desafio e depois exporte para enviar!

Concluindo todos os passos informados, basta salvar o arquivo .ipynb do notebook e zipar juntamente com os parquet das tabelas, postar no seu github e enviar o link para ricardo.suhete@autoglass.com.br

A entrega da tarefa termina 7 dias após o recebimento deste notebook. Após o prazo será entendindo que o condidato desistiu da vaga.



# Instalação dos pacotes necessários

In [ ]:
!pip install -q findspark
!pip install -q unidecode
!pip install -q pyspark
import findspark
findspark.init()

import requests as r
import json
import unidecode
from pyspark.sql import SparkSession, Row
from pyspark.sql import functions as F

from pyspark.sql.types import StructField, StructType, StringType, MapType, ArrayType, IntegerType, BooleanType

import urllib.parse

spark = SparkSession.builder \
      .master("local[1]") \
      .appName("Teste_Spark_Autoglass_candidatos") \
      .getOrCreate()

# Variáveis

In [2]:
code = "AQAWqugPoZceiqfhc-cQJGWo5x6EYpv9EiSCbqV3wlzPcpEOU61bN1PcEMzWdYfK-GVS9JMglUU6sY_vhYK1Jm12k7VlwtTh4Y07t2MCpHA29qR2XjYZUO5d4wNdZkn5fBOEZwN4QLJmy5Z21PGgxqr7LYLWj2Zf4x0je1P_clmaFLa0J_XOpEezA8KShp8Bl9XS5ITzyqIqvRTtIl5h1GQO_QDkkA"

CLIENT_ID = "377e2eedd81c42c085d6e4ee8275ee1b"

AUTH_URL = "https://accounts.spotify.com/api/authorize"
TOKEN_URL = "https://accounts.spotify.com/api/token"
REDIRECT_URL = "http://localhost:8000/callback"

API_BASE_URL = "https://api.spotify.com/v1/"

scope = "user-read-private user-read-email"

## Schemas

In [3]:
schema_playlist = StructType([
    StructField("href", StringType(), True),
    StructField("limit", IntegerType(), True),
    StructField("next", StringType(), True),
    StructField("offset", IntegerType(), True),
    StructField("previous", StringType(), True),
    StructField("total", IntegerType(), True),
    StructField("items", ArrayType(
        StructType([
            StructField("collaborative", BooleanType(), True),
            StructField("description", StringType(), True),
            StructField("external_urls", MapType(StringType(), StringType()), True),
            StructField("href", StringType(), True),
            StructField("id", StringType(), True),
            StructField("images", ArrayType(
                StructType([
                    StructField("url", StringType(), True),
                    StructField("height", StringType(), True),
                    StructField("width", StringType(), True),
                ])
            ), True),
            StructField("name", StringType(), True),
            StructField("owner", StructType([
                StructField("external_urls", MapType(StringType(), StringType()), True),
                StructField("href", StringType(), True),
                StructField("id", StringType(), True),
                StructField("type", StringType(), True),
                StructField("uri", StringType(), True),
                StructField("display_name", StringType(), True),
            ]), True),
            StructField("public", BooleanType(), True),
            StructField("snapshot_id", StringType(), True),
            StructField("tracks", StructType([
                StructField("href", StringType(), True),
                StructField("total", IntegerType(), True),
            ]), True),
            StructField("type", StringType(), True),
            StructField("uri", StringType(), True),
            StructField("primary_color", StringType(), True),
        ])
    ), True),
])

schema_artistas = StructType([
    StructField("name", StringType(), True),
    StructField("href", StringType(), True),
    StructField("id", StringType(), True),
    StructField("popularity", IntegerType(), True),
    StructField("type", StringType(), True),
    StructField("uri", StringType(), True),
    StructField("external_urls", StructType([
        StructField("spotify", StringType(), True)
    ]), True),
    StructField("followers", StructType([
        StructField("href", StringType(), True),
        StructField("total", IntegerType(), True)
    ]), True),
    StructField("images", ArrayType(StructType([
        StructField("url", StringType(), True),
        StructField("height", IntegerType(), True),
        StructField("width", IntegerType(), True)
    ])), True),
    StructField("genres", ArrayType(StringType()), True)
])



# Autenticação

In [4]:
token_headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}

token_data = {
    "grant_type": "authorization_code",
    "code": code,
    "redirect_uri": REDIRECT_URL
}

response = r.post(TOKEN_URL, data=token_data, headers=token_headers,auth=('377e2eedd81c42c085d6e4ee8275ee1b', '1ac2c7b464ed4201886b697cbd4d26e7'))

ACESS_TOKEN = response.json()['access_token']

In [ ]:
url = "https://accounts.spotify.com/api/token"
payload = "grant_type=client_credentials"

headers = {
    "Content-Type" : "application/x-www-form-urlencoded"
}

response = r.post(
    url,
    headers=headers,
    data=payload,
    auth=('377e2eedd81c42c085d6e4ee8275ee1b', '1ac2c7b464ed4201886b697cbd4d26e7'))

print(response.json())
access_token = response.json()['access_token']
access_token

# Chamadas API

## Rock Nacional; Piseiro/Arrocha; Pop Internacional

In [ ]:
## Adicionar for com outros estilos na busca

request_url = "search"
offset = 0
artistas = []

headers = {
    "Authorization" : f"Bearer {access_token}"
}

generos = ["Rock Nacional", "Piseiro/Arrocha", "Pop Internacional"]
for genero in generos:
  params = {
      "q": f"genre: {genero}",
      "type": "artist",
      "market": "BR"
  }

  encoded_params = urllib.parse.urlencode(params)
  url = f"{API_BASE_URL}{request_url}?{encoded_params}"
  response = r.get(url, headers=headers)
  data = response.json()
  artists = data.get('artists', {}).get('items', [])
  artistas.extend(artists)
  print(len(artistas))

df_artistas = spark.createDataFrame(artistas, schema_artistas)

df_artistas.show(50)


## Get Artist

In [7]:
base_url = "https://api.spotify.com/v1/"

request_url = "artists/"

id = "1YOVBTvznjiDvtAj4ExHeo"

url = f"{base_url}{request_url}{id}"

headers = {
    "Authorization" : f"Bearer {access_token}"
}

response = r.get(url, headers=headers)
print(response.json())

## Get Artist's Albums

In [8]:
#Request para o endpoint Get Album Tracks utilizando os album id do dataframe anterior

album_id = ""

request_url = f"/albums/{album_id}"

url = f"{API_BASE_URL}{request_url}"

headers = {
    "Authorization" : f"Bearer {ACESS_TOKEN}"
}

response = r.get(url, headers=headers)
data = [response.json()]

## Get Album Tracks

In [ ]:
#Request para o endpoint Get Album Tracks utilizando os album id do dataframe anterior

album_id = ""

request_url = f"/albums/{album_id}/tracks"

url = f"{API_BASE_URL}{request_url}"

headers = {
    "Authorization" : f"Bearer {ACESS_TOKEN}"
}

response = r.get(url, headers=headers)
data = [response.json()]

## Get Current User's Playlists

In [15]:
#Request para o endpoint Get Current User's Playlists
request_url = "me/playlists"


url = f"{API_BASE_URL}{request_url}"

headers = {
    "Authorization" : f"Bearer {ACESS_TOKEN}"
}

response = r.get(url, headers=headers)
data = [response.json()]

In [ ]:
df_user_playlists = spark.createDataFrame(data, schema=schema_playlist)
df_user_playlists = df_user_playlists.select("href", "limit", "next", "offset", "previous", "total", F.explode("items").alias("item"))
df_user_playlists = df_user_playlists.select("href", "limit", "next", "offset", "previous", "total", "item.*")

df_user_playlists.show(truncate=False)

In [14]:
#Criação da TempView; TempTable está depreciado nas versões mais recentes

df_user_playlists.createOrReplaceTempView("User_Playlists")